In [15]:
# Getting the vehicules trajectory completion by the number of shapes traveled and the number of shapes planned

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing required packages for this notebook session
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/8f/4a/16ffdfc33d93f02604ae9ed1ddb6369030b6f61b583f31dc84e0d0da05c1/botocore-1.19.47-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/71/45d36a8df68f3ebb098d6861b2c017f3d094538c0fb98fa61d4dc43e69b9/urllib3-1.26.2-py2.py3-none-any.whl

In [ ]:
import pyspark.sql.functions as f
import boto3 

shapes = spark.read.csv("s3://mobility-traces-sp/aux-files/gtfs/shapes.csv",header=True)
shape_df = shapes.groupBy("shape_id").agg(f.countDistinct("shape_pt_sequence").alias("shape_size"))

csv_out = "day,mean,min,max,stddev,quantile_25,quantile_50,quantile_75\n"

for day in range(1,32):
    # reading map matching files
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510"+ str(day) + "/")
    

    # counting the number of different shape sequences for a bus in a day
    shape_count = traces.groupBy("id_avl","line_id").agg(f.countDistinct("min_shape_sequence").alias("shape_seq_count"))
    
    # counting the number of traces for a bus in a day
    trace_count = traces.groupBy("id_avl","line_id").agg(f.count("id_avl").alias("traces_count_per_bus"))
    
    trace_shape = trace_count.join(shape_count,on=["id_avl","line_id"],how="left")
    trace_shape_with_shape_id = trace_shape.join(traces,on=["id_avl","line_id"],how="left").dropDuplicates(subset = ["id_avl","line_id"]).select("id_avl","line_id","shape_id","shape_seq_count","traces_count_per_bus")

    trace_shape_with_size = trace_shape_with_shape_id.join(shape_df, on=["shape_id"],how="left")
    df = trace_shape_with_size.withColumn("completion_trip",(f.col("shape_seq_count")/f.col("shape_size") * 100))
    stats = df.agg(f.mean('completion_trip').alias('mean'),
                       f.min('completion_trip').alias('min'),
                       f.max('completion_trip').alias('max'),
                       f.stddev('completion_trip').alias("stddev")).collect()
    
    quantiles = df.approxQuantile("completion_trip", [0.25,0.5,0.75], 0.0001)
    
    csv_out += "MO_1510" + str(day) +","+ str(stats[0]["mean"]) +","+ str(stats[0]["min"]) + "," + str(stats[0]["max"]) + "," + str(stats[0]["stddev"]) + "," +str(quantiles[0]) + "," +str(quantiles[1]) +"," +str(quantiles[2]) + "\n"


import boto3   
s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/9-trip-completion.csv')



In [ ]:
# Saving counting for shapes and traces for each bus
import pyspark.sql.functions as f

shapes = spark.read.csv("s3://mobility-traces-sp/aux-files/gtfs/shapes.csv",header=True)
shape_df = shapes.groupBy("shape_id").agg(f.countDistinct("shape_pt_sequence").alias("shape_size"))

csv_out = "day,mean,min,max,stddev,quantile_25,quantile_50,quantile_75\n"

for day in range(1,32):
    # reading map matching files
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510"+ str(day) + "/")
    

    # counting the number of different shape sequences for a bus in a day
    shape_count = traces.groupBy("id_avl","line_id").agg(f.countDistinct("min_shape_sequence").alias("shape_seq_count"))
    
    # counting the number of traces for a bus in a day
    trace_count = traces.groupBy("id_avl","line_id").agg(f.count("id_avl").alias("traces_count_per_bus"))
    
    trace_shape = trace_count.join(shape_count,on=["id_avl","line_id"],how="left")
    trace_shape_with_shape_id = trace_shape.join(traces,on=["id_avl","line_id"],how="left").dropDuplicates(subset = ["id_avl","line_id"]).select("id_avl","line_id","shape_id","shape_seq_count","traces_count_per_bus")

    trace_shape_with_size = trace_shape_with_shape_id.join(shape_df, on=["shape_id"],how="left")
    df = trace_shape_with_size.withColumn("completion_trip",(f.col("shape_seq_count")/f.col("shape_size") * 100))
   
    df.write.parquet("s3://mobility-traces-sp/statistics/buses_count_shape_traces/MO_1510"+ str(day) + "/")